In [1]:
from astropy.io import ascii
from astropy.table import Table,QTable
import numpy as np
import pandas as pd

In [25]:
iso_mini_fits = Table.read('iso_mini_new.csv',format='ascii.csv')

In [28]:
galaxy_ID_new = []
rho0_b_new = []
Rb_new = []
SigD_new = []
Rd_new = []
rho0_h_new = []
Rh_new = []
incl_new = []
phi_new = []
x_cen_new = []
y_cen_new = []
Vsys_new = []
chi2_new = []
for i in range(len(iso_mini_fits)):
    if not np.isnan(iso_mini_fits['chi2'][i]):
        galaxy_ID_new.append(iso_mini_fits['galaxy_ID'][i])
        rho0_b_new.append(iso_mini_fits['rho0_b'][i])
        Rb_new.append(iso_mini_fits['Rb'][i])
        SigD_new.append(iso_mini_fits['SigD'][i])
        Rd_new.append(iso_mini_fits['Rd'][i])
        rho0_h_new.append(iso_mini_fits['rho0_h'][i])
        Rh_new.append(iso_mini_fits['Rh'][i])
        incl_new.append(iso_mini_fits['incl'][i])
        phi_new.append(iso_mini_fits['phi'][i])
        x_cen_new.append(iso_mini_fits['x_cen'][i])
        y_cen_new.append(iso_mini_fits['y_cen'][i])
        Vsys_new.append(iso_mini_fits['Vsys'][i])
        chi2_new.append(iso_mini_fits['chi2'][i])


In [31]:
c_iso = Table()
c_iso['galaxy_ID'] = galaxy_ID_new
c_iso['rho0_b'] = rho0_b_new
c_iso['Rb'] = Rb_new
c_iso['SigD'] = SigD_new
c_iso['Rd'] = Rd_new
c_iso['rho0_h'] = rho0_h_new
c_iso['Rh'] = Rh_new
c_iso['incl'] = incl_new
c_iso['phi'] = phi_new
c_iso['x_cen'] = x_cen_new
c_iso['y_cen'] = y_cen_new
c_iso['Vsys'] = Vsys_new
c_iso['chi2'] = chi2_new
c_iso.write('iso_mini_clean.csv', format='ascii.csv', overwrite=True)

In [32]:
c_iso

galaxy_ID,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-12701,0.528291628,0.519199474,1186.662904,6.27584858,-0.632613278,1.775568522,0.367529844,1.064241938,37.90950146,39.38451951,2.597767546,34.01136626
10001-12702,0.19485593,0.40910858,540.5425784,16.36982311,-2.44486967,0.100065559,0.676515511,3.327074163,37.40979448,36.46819334,3.682228086,5.870467141
10001-12703,0.854751128,0.561502924,432.3613461,13.46075637,-6.999999834,813.4646417,0.602870445,0.918460484,37.67294365,38.07749965,13.7671227,154.109528
10001-12704,-0.315150165,8.942796053,738.1773728,18.67724812,-0.186955875,3.615596829,0.569485607,0.60289846,33.77403134,31.2206351,-7.589383346,436.9663356
10001-12705,0.999956174,2.328231207,2999.999996,29.99996632,1.999946529,0.187924306,0.118562599,5.290620659,36.73928252,36.85037884,-20.55299729,44.51862361
10001-1901,1.0,4.104889409,2977.80581,21.68738464,1.658863469,1.263754815,0.104787037,4.717071664,15.58168468,26.16208213,-19.0329717,22.78812179
10001-3702,-2.754512592,0.260737874,574.8419473,18.99880064,-0.965736351,0.480721511,0.546396962,2.30556051,22.4800498,22.36719729,-0.54125803,10.94606125
10001-3703,1.0,5.352876893,1242.536506,6.287390323,1.008079284,0.915433029,0.100655462,6.411034013,23.38469674,20.72803333,-13.14915792,85.79243466
10001-6102,0.595329696,0.699041976,0.100164626,29.99996174,-1.287456859,999.9999998,1.01969221,4.123084428,26.20185798,27.90454897,-19.13321161,210.8509922


In [33]:
nfw_mini_fits = Table.read('nfw_mini_new.csv',format='ascii.csv')

In [34]:
galaxy_ID_new = []
rho0_b_new = []
Rb_new = []
SigD_new = []
Rd_new = []
rho0_h_new = []
Rh_new = []
incl_new = []
phi_new = []
x_cen_new = []
y_cen_new = []
Vsys_new = []
chi2_new = []
for i in range(len(nfw_mini_fits)):
    if not np.isnan(nfw_mini_fits['chi2'][i]):
        galaxy_ID_new.append(nfw_mini_fits['galaxy_ID'][i])
        rho0_b_new.append(nfw_mini_fits['rho0_b'][i])
        Rb_new.append(nfw_mini_fits['Rb'][i])
        SigD_new.append(nfw_mini_fits['SigD'][i])
        Rd_new.append(nfw_mini_fits['Rd'][i])
        rho0_h_new.append(nfw_mini_fits['rho0_h'][i])
        Rh_new.append(nfw_mini_fits['Rh'][i])
        incl_new.append(nfw_mini_fits['incl'][i])
        phi_new.append(nfw_mini_fits['phi'][i])
        x_cen_new.append(nfw_mini_fits['x_cen'][i])
        y_cen_new.append(nfw_mini_fits['y_cen'][i])
        Vsys_new.append(nfw_mini_fits['Vsys'][i])
        chi2_new.append(nfw_mini_fits['chi2'][i])


In [35]:
c_nfw = Table()
c_nfw['galaxy_ID'] = galaxy_ID_new
c_nfw['rho0_b'] = rho0_b_new
c_nfw['Rb'] = Rb_new
c_nfw['SigD'] = SigD_new
c_nfw['Rd'] = Rd_new
c_nfw['rho0_h'] = rho0_h_new
c_nfw['Rh'] = Rh_new
c_nfw['incl'] = incl_new
c_nfw['phi'] = phi_new
c_nfw['x_cen'] = x_cen_new
c_nfw['y_cen'] = y_cen_new
c_nfw['Vsys'] = Vsys_new
c_nfw['chi2'] = chi2_new
c_nfw.write('nfw_mini_clean.csv', format='ascii.csv', overwrite=True)

In [36]:
c_nfw

galaxy_ID,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-12701,0.733473448,0.495600618,2999.998433,3.111775269,-4.074551898,303.9545695,0.284735707,1.061849139,38.08488823,39.50912377,1.934119232,41.33709406
10001-12702,-1.067656156,0.583187132,373.8127075,22.53772179,-3.445389625,380.8128713,0.575517189,3.225688008,38.7448229,37.57976068,-3.099548616,6.581584816
10001-12703,0.999999139,2.613756872,2999.999944,29.99999408,1.525784259,1.014109964,0.107683759,0.907028069,37.97623288,38.10905933,10.2378397,50.67156935
10001-12704,-0.394083478,1.518297612,1849.756213,24.03794764,0.331708301,0.100037181,0.623673648,1.44887249,36.80262707,37.63929818,2.638843591,32.95528142
10001-12705,0.934564241,2.62857121,3000.0,29.89088204,1.54698321,0.920046942,0.110250275,5.336285793,37.3088779,36.75879075,-15.09701935,41.11414574
10001-1901,0.999956174,4.999963607,2999.994478,4.07919604,-2.413545292,945.3765059,0.1368385,4.842025325,17.01330068,18.83220905,3.816145089,20.67143069
10001-3702,-2.256590304,2.187998866,122.8218951,29.93233374,-3.884328559,432.8205053,0.519986069,2.297406455,22.79146827,22.15863061,-0.984236708,10.72675649
10001-3703,0.935341816,4.881705665,2999.999944,2.381822753,-6.998522261,0.100004088,0.134822808,3.00650193,31.99995965,21.1208082,-2.535829186,6856.964488
10001-6102,0.870986292,0.620977965,3000.0,8.061516521,-6.999949834,0.100045078,0.513538661,4.089468345,26.17048942,28.33103695,-21.84053603,300.7409233


In [37]:
bur_mini_fits = Table.read('bur_mini_new.csv',format='ascii.csv')

In [38]:
galaxy_ID_new = []
rho0_b_new = []
Rb_new = []
SigD_new = []
Rd_new = []
rho0_h_new = []
Rh_new = []
incl_new = []
phi_new = []
x_cen_new = []
y_cen_new = []
Vsys_new = []
chi2_new = []
for i in range(len(bur_mini_fits)):
    if not np.isnan(bur_mini_fits['chi2'][i]):
        galaxy_ID_new.append(bur_mini_fits['galaxy_ID'][i])
        rho0_b_new.append(bur_mini_fits['rho0_b'][i])
        Rb_new.append(bur_mini_fits['Rb'][i])
        SigD_new.append(bur_mini_fits['SigD'][i])
        Rd_new.append(bur_mini_fits['Rd'][i])
        rho0_h_new.append(bur_mini_fits['rho0_h'][i])
        Rh_new.append(bur_mini_fits['Rh'][i])
        incl_new.append(bur_mini_fits['incl'][i])
        phi_new.append(bur_mini_fits['phi'][i])
        x_cen_new.append(bur_mini_fits['x_cen'][i])
        y_cen_new.append(bur_mini_fits['y_cen'][i])
        Vsys_new.append(bur_mini_fits['Vsys'][i])
        chi2_new.append(bur_mini_fits['chi2'][i])

In [39]:
c_bur = Table()
c_bur['galaxy_ID'] = galaxy_ID_new
c_bur['rho0_b'] = rho0_b_new
c_bur['Rb'] = Rb_new
c_bur['SigD'] = SigD_new
c_bur['Rd'] = Rd_new
c_bur['rho0_h'] = rho0_h_new
c_bur['Rh'] = Rh_new
c_bur['incl'] = incl_new
c_bur['phi'] = phi_new
c_bur['x_cen'] = x_cen_new
c_bur['y_cen'] = y_cen_new
c_bur['Vsys'] = Vsys_new
c_bur['chi2'] = chi2_new
c_bur.write('bur_mini_clean.csv', format='ascii.csv', overwrite=True)

In [40]:
c_bur

galaxy_ID,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
10001-12701,0.68155474,0.638786013,1860.291507,20.42483769,-0.789535717,19.24848673,0.307073372,1.067543677,37.84132041,39.03579917,1.355036259,33.89159087
10001-12702,0.20844105,0.381786517,599.4433204,16.96940508,-2.719274013,0.100161407,0.664441233,3.286380151,37.82730585,36.77274051,1.571576218,6.043839703
10001-12703,0.613635106,0.846027509,1219.234831,18.77163674,-1.034386038,12.29096582,0.387023788,0.936038214,37.17770907,37.14990892,10.29870493,47.67803286
10001-12704,-0.426488624,3.377615302,1347.583716,18.8655543,0.215368919,0.420883043,0.525174822,1.500358851,36.17773962,35.39972504,3.109231898,34.84223033
10001-12705,0.997148194,2.541336061,2999.999944,29.99754727,1.999946529,0.549033901,0.108542837,5.308450866,37.23893729,36.9091286,-15.13700499,36.02966614
10001-1901,0.999956174,0.501445561,2999.999956,9.050518147,1.51248123,3.404886578,0.10163082,4.714187599,17.52475396,19.41632326,7.264201096,24.31812135
10001-3702,-3.95294298,0.201992003,403.129002,12.68648851,-4.467995935,1.063722712,0.656174217,2.326174231,22.71271086,21.94141491,0.25057755,14.36108846
10001-3703,0.998867724,4.842892294,2999.999944,0.359520474,0.415542904,55.16004938,0.1,3.45426448,31.6208928,23.77080303,24.41716881,5860.962195
10001-6102,0.585612291,0.709487518,2188.071336,29.99482558,-1.602365903,0.108098384,0.343656595,2.608376814,36.27683068,27.98299829,-22.19688648,10344.11607
